In [11]:
%%capture o
!cd build && make

In [12]:
import os
import subprocess
import numpy as np

In [13]:
args = ("./build/micro_aevol_cpu")

In [14]:
%%timeit -n 2 -r 5 -o 
popen = subprocess.Popen(args, stdout=subprocess.PIPE)
popen.wait()

2 loops, best of 5: 1.61 s per loop


<TimeitResult : 2 loops, best of 5: 1.61 s per loop>

In [15]:
res = _
runs = np.array(res.all_runs) / res.loops
print(runs)

[1.73761897 1.65931401 1.65083684 1.61071869 1.6206061 ]
